# Terrain

> Create a TIN terrain from point height data

In [ ]:
#| default_exp terrain

In [ ]:
#| hide
from pprint import pprint
from nbdev.showdoc import *
from fastcore.basics import patch

In [ ]:
#| export
import os.path
import subprocess
import math
from collections import Counter

import fiona
from fiona import Geometry, Feature, Properties
import numpy as np
import matplotlib.tri as tri
from matplotlib.pyplot import tricontour
from shapely.geometry import Point, LineString, Polygon
from shapely.geometry import mapping, shape
from rtree import index
from tqdm import tqdm

from cdBoundary.boundary import ConcaveHull

In [ ]:
#| export
class Terrain:

    ''' Class representing a TIN terrain '''

    def __init__(self, surface: str) -> None:

        if surface[-5:] != '.gpkg':
            self.surface = surface + '.gpkg'
        else:
            self.surface = surface

        self.trianglebin = None
        self.model = None
        self.crs = None
        create_pts_layer = False
        file_exists = os.path.isfile(self.surface)
        if file_exists:
            layers = fiona.listlayers(self.surface)
            if 'Points' not in layers:
                create_pts_layer = True
            else:
                with fiona.open(self.surface, layer='Points') as source:
                    self.crs = source.crs
        
        if not file_exists or create_pts_layer:
            driver = 'GPKG'
            schema = {'geometry': 'Point',
                      'properties': {'No': 'int',
                                     'X': 'float',
                                     'Y': 'float',
                                     'Elevation': 'float',
                                     'Comment': 'str',
                                     'Comment2': 'str',
                                     'Contourable': 'int'}}
            with fiona.open(self.surface, 'w', layer='Points', 
                            driver=driver, crs=self.crs, 
                            schema=schema) as sink:
                pass
        

Method to load the survey points.

'''

Loads the points from a Fiona supported format.
    
            Parameters:
                pointsfile (str)   : A Fiona supporte file format
                in_layer (str)     : Layer (If applicable) where to points 
                                     are located in the file
                z_field (str)      : The field in the attribute table
                                     containing the elevation
                tag (str)          : The field in the attribute table
                                     containing the point comment
                overwrite (bool)   : If `False` and there is existing points in the
                                     layer the method will terminate. Set to
                                     `True` to load the new points. If the crs of
                                     the new points is None the old crs will be used
                                     otherwise the crs will also be overwritten
                use_z_coord (bool) : Use the z coordinate from the geometry to
                                     obtain the elevation
                tol (float)        : Any points closer than this value to
                                     existing points will be loaded but will
                                     be made non-contourable

            Returns:
                None       
'''

In [ ]:
#| export
@patch
def loadpoints(self:Terrain, pointsfile: str, layer: str=None,
               z_field: str=None, tag: str='Comment', overwrite: bool=False,
               use_z_coord: bool=False, tol: float=0.01) -> None:

    ''' Loads the points from a Fiona supported format.
    
            Parameters:
                pointsfile (str)   : A Fiona supporte file format
                layer (str)        : Layer (If applicable) where to points 
                                     are located in the file
                z_field (str)      : The field in the attribute table
                                     containing the elevation
                tag (str)          : The field in the attribute table
                                     containing the point comment
                overwrite (bool)   : If `False` and there is existing points in the
                                     layer the method will terminate. Set to
                                     `True` to load the new points. If the crs of
                                     the new points is None the old crs will be used
                                     otherwise the crs will also be overwritten
                use_z_coord (bool) : Use the z coordinate from the geometry to
                                     obtain the elevation
                tol (float)        : Any points closer than this value to
                                     existing points will be loaded but will
                                     be made non-contourable

            Returns:
                None       
    '''

    # Todo: Specify a schema in the parameters for custom schemas
    
    with fiona.open(self.surface, layer='Points') as source:
        if len(source) > 0 and overwrite is False:
            raise Exception('''There is existing points in the layer.
                               Please set `overwrite` to True to replace''')

    # If `z_field` and/or `tag` has no match the program attempts to find one
    with fiona.open(pointsfile, layer=layer) as source:
        infields = list(source.schema['properties'].keys())
        for feat in source:
            geom = shape(feat.geometry)
            hasz = geom.has_z
            break
    
    z_options = ['z', 'elevation', 'height']
    tag_options = ['comment', 'descriptio', 'description', 'note', 'tag']
    if z_field is not None:
        z_options.insert(0, z_field)
    tag_options.insert(0, tag)

    if z_field not in infields and use_z_coord is False:
        for col in infields:
            for item in z_options:
                if col.casefold() == item:
                    z_field = col
                    break
    if tag not in infields or tag is None:
            for col in infields:
                for item in tag_options:
                    if col.casefold() == item:
                        tag = col
                        break

    schema = {'geometry': 'Point',
              'properties': {'No': 'int',
                             'X': 'float',
                             'Y': 'float',
                             'Elevation': 'float',
                             'Comment': 'str',
                             'Comment2': 'str',
                             'Contourable': 'bool'}}

    ptdata = list()
    print('Reading points ...')
    print('Checking for duplicate points. `tol=', str(tol), '`', sep='')
    with fiona.open(pointsfile, layer=layer) as source:
        self.crs = source.crs
        idx = index.Index()
        for i, pt in enumerate(tqdm(source, bar_format='{l_bar}{bar:50}{r_bar}{bar:-50b}')):
            point = shape(pt.geometry)
            props = dict(pt.properties)
            if z_field is not None:
                z = float(props[z_field])
            else:
                z = point.z
            pt3d = Point(point.x, point.y, z)
            rec = dict()
            rec['No'] = i+1
            rec['X'] = round(point.x, 3)
            rec['Y'] = round(point.y, 3)
            rec['Elevation'] = round(z, 3)
            rec['Comment'] = props[tag]
            if 'Comment2' in infields:
                rec['Comment2'] = str(props['Comment2'])
            else:
                rec['Comment2'] = None
            if props[z_field] == -999 and z_field is not None:
                rec['Contourable'] = False
            else:
                rec['Contourable'] = True
            hits = list(idx.intersection(point.buffer(tol).bounds))
            flag = True
            for hit in hits:
                point2 = shape(ptdata[hit].geometry)
                if point.distance(point2) <= tol:
                    rec['Contourable'] = False
                    if rec['Comment2'] is None:
                        rec['Comment2'] = 'Duplicate'
                    flag = False
                    break
            feat = Feature(geometry = Geometry.from_dict(mapping(pt3d)), 
                           properties = Properties.from_dict(rec))
            ptdata.append(feat)
            if flag:
                idx.insert(i, (point.bounds))

    print()
    print('Writing points to file ...')
    with fiona.open(self.surface, 'w', layer='Points',
                    crs=self.crs, driver='GPKG', schema=schema) as sink:
        if len(ptdata) < 1000:
            sink.writerecords(ptdata)
        else:
            batchdiv = 100
            batches = self._chunks(ptdata, batchdiv)
            for batch in tqdm(batches, total=batchdiv, 
                              bar_format='{l_bar}{bar:50}{r_bar}{bar:-50b}'):
                sink.writerecords(batch)
        
    # Add a blank 'Breaklines' layer if 'Features' layer does not exist
    if 'Breaklines' not in fiona.listlayers(self.surface):
        
        schema = {'geometry': 'LineString',
                  'properties': {'Code': 'str',
                                 'Name': 'str',
                                 'Length': 'float',
                                 'Vertices': 'int',
                                 'Type': 'bool'}}
        
        with fiona.open(self.surface, 'w', layer='Breaklines', 
                        driver='GPKG', crs=self.crs, schema=schema) as sink:
            pass
            
    print('Done.')

In [ ]:
#| export
@patch
def add_breaklines(self:Terrain, file: str, layer: str=None, 
                   snap: bool=True, snaptol: float=0.05,
                   append: bool=True) -> None:

    ''' Using a Fiona supported file format of LineStrings to
        load strings representing breaklines.
    '''

    points, idx = self._update_point_index()

    schema = {'geometry': 'LineString',
              'properties': {'Code': 'str',
                             'Name': 'str',
                             'Length': 'float',
                             'Vertices': 'int',
                             'Type': 'bool'}}

    # Add the new features
    # Todo: Handle MultilineStrings
    blinedb = list()
    with fiona.open(file, layer=layer) as source:
        infields = list(source.schema['properties'].keys())
        name = False
        for i, field in enumerate(infields):
            if field.lower() == 'name':
                name = True
                name_idx = i
        for feat in source:
            geom = shape(feat.geometry)
            coords = list(geom.coords)
            code = list()
            for i, pt in enumerate(coords):
                x, y = pt[0], pt[1]
                currentpt = list(idx.nearest((x, y, x, y), 1))[0]
                cprops = dict(points[currentpt].properties)
                tag = points[currentpt]['properties']['Comment']
                code.append(tag)
            datatmp = Counter(code)
            fcode = max(code, key=datatmp.get)
            rec = dict()
            rec['Code'] = fcode
            props = dict(feat.properties)
            if name:
                rec['Name'] = props[infields[name_idx]]   
            else:
                rec['Name'] = None
            rec['Length'] = round(geom.length, 3)
            rec['Vertices'] = len(list(geom.coords))
            rec['Type'] = 'Soft'
            feature = Feature(geometry = feat.geometry, 
                              properties = Properties.from_dict(rec))
            blinedb.append(feature)

    if append:
        mode = 'a'
    else:
        mode = 'w'

    with fiona.open(self.surface, mode, layer='Breaklines',
                    driver='GPKG', crs=self.crs, schema=schema) as sink:
        sink.writerecords(blinedb)

In [ ]:
#| export
@patch
def boundary(self:Terrain, tol: float=50) -> None:

    '''
    Calculates the concave hull of the points.
    This is called "join edge points" in Highroad
    terrain is the geospatial file containing the points
    The "tol" is the length of the largest outside edge in your units
    '''

    ch = ConcaveHull()
    points = list()
    with fiona.open(self.surface, layer='Points') as source:
        for feat in source:
            props = dict(feat.properties)
            if props['Contourable']:
                geom = shape(feat.geometry)
                x, y = geom.x, geom.y
                z = props['Elevation']
                points.append((x, y, z))

    ch.loadpoints(points)
    ch.calculatehull(tol=tol)

    schema = {'geometry': 'Polygon', 
              'properties': {'Area': 'float',
                             'Perimeter': 'float',
                             'Vertices': 'int'}}
    with fiona.open(self.surface, 'w', layer='Boundary',
                    crs=self.crs, driver='GPKG', schema=schema) as sink:
        props = dict()
        props['Area'] = round(ch.hull.area, 3)
        props['Perimeter'] = round(ch.hull.length, 3)
        props['Vertices'] = len(list(ch.hull.exterior.coords))
        feature = Feature(geometry = Geometry.from_dict(mapping(ch.hull)), 
                          properties = Properties.from_dict(props))
        sink.write(feature)

In [ ]:
#| export
@patch
def boundary_estimate(self:Terrain, perc: float=95) -> float:

    ''' Gives an estimate of the val to be used in `tol` for `calculatehull`
        to yeild a nice looking concave hull  

            Parameters:  
                perc (float) : Number between 0 and 100

            Returns:  
                (float) : Suggested value to use as `tol` in
                          `calculatehull`
    '''

    ch = ConcaveHull()
    points = list()
    with fiona.open(self.surface, layer='Points') as source:
        for feat in source:
            props = dict(feat.properties)
            if props['Contourable']:
                geom = shape(feat.geometry)
                x, y = geom.x, geom.y
                z = props['Elevation']
                points.append((x, y, z))

    ch.loadpoints(points)
    estimate = ch.estimate(perc=perc)

    return estimate

In [ ]:
#| export
@patch
def _generator_function(self:Terrain, geometries: list) -> tuple:
    ''' rtree bulk loading '''
    for i, geom in enumerate(geometries):
        yield (i, (shape(geom.geometry).bounds), geom)

In [ ]:
#| export
@patch
def _update_point_index(self:Terrain, ignore: bool=True) -> tuple:

    with fiona.open(self.surface, layer='Points') as source:
        points = list()
        for pt in source:
            props = dict(pt.properties)
            if props['Contourable'] is False and ignore is True:
                continue
            points.append(pt)

    ptidx = index.Index(self._generator_function(points))

    return (points, ptidx)

Got the *recipe* for dividing lists into chunks from [https://stackoverflow.com/questions/24483182/python-split-list-into-n-chunks](https://stackoverflow.com/questions/24483182/python-split-list-into-n-chunks)

In [ ]:
#| export
@patch
def _chunks(self:Terrain, lst: list, n: int):
    ''' Yield n number of sequential chunks from lst '''
    d, r = divmod(len(lst), n)
    for i in range(n):
        si = (d+1)*(i if i < r else r) + d*(0 if i < r else i - r)
        yield lst[si:si+(d+1 if i < r else d)]

In [ ]:
def line2segments(curve):
    return list(map(LineString, zip(curve.coords[:-1], curve.coords[1:])))
    

def geom2segments(geometry):

    if geometry.geom_type == 'LineString' or geometry.geom_type == 'LinearRing':
        return(line2segments(geometry))
    elif geometry.geom_type == 'Polygon':
        seglist = line2segments(LinearRing(list(geometry.exterior.coords)))
        for interior in geometry.interiors:
            poly = Polygon(interior)
            seglist += line2segments(LinearRing(list(poly.exterior.coords)))
        return seglist
    # Todo: Implement Multipart geometries (Easy)

**Todo**

- ~~Concave Hull~~
- ~~Hull estimate~~
- ~~Optimise `loadpoints`. Use the `generator` function.~~ No speedup
- Allow for option not to check for duplicates
- Isolated function to check for duplicates afterwards
- Layer Report
    - No
    - Layer name
    - Feature Count
- Snap and drape breaklines
- Drape hard breaklines
- Import hard breaklines
- Merge lines
- Split lines into segments
- Layer Styles

## Testing -------------------------

In [ ]:
!ls ../../testing

Bambi.gpkg  Bluegum.gpkg  GIS


In [ ]:
test2 = Terrain('../../testing/Bluegum')
ptsource = '/home/andre/Documents/Jobs/Bluegum/Data/Terrain/bg_points_test.gpkg'
test2.loadpoints(ptsource, layer='Points', overwrite=True)

Reading points ...
Checking for duplicate points. `tol=0.01`


100%|██████████████████████████████████████████████████| 234427/234427 [01:00<00:00, 3883.63it/s]                                                                                                                                                             



Writing points to file ...


100%|██████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.11it/s]                                                                                                                                                                     


Done.


In [ ]:
with fiona.open('../../testing/Bluegum.gpkg', layer='Points') as source:
    print(len(source))
    for feat in source:
        props = dict(feat.properties)
        print()
        pprint(props)
        break

234427

{'Comment': 'cloud',
 'Comment2': '',
 'Contourable': True,
 'Elevation': 106.122,
 'No': 1,
 'X': -78872.428,
 'Y': -3759904.426}


### Bambi Test Data

In [ ]:
bambi = Terrain('../../testing/Bambi')
ptsource = '/home/andre/Documents/Jobs/1280 - Lydenburg/culvertdata/data/processed/surveypts1_mod.shp'
bambi.loadpoints(ptsource, overwrite=True, z_field='Height', tol=0.001)

Reading points ...
Checking for duplicate points. `tol=0.001`


100%|██████████████████████████████████████████████████| 12770/12770 [00:03<00:00, 4102.31it/s]                                                                                                                                                               



Writing points to file ...


100%|██████████████████████████████████████████████████| 100/100 [00:00<00:00, 214.47it/s]                                                                                                                                                                    


Done.


In [ ]:
%%time

blines = '/home/andre/Documents/Jobs/1280 - Lydenburg/culvertdata/data/processed/Breakline Edges Edit.shp'
bambi.add_breaklines(blines, append=False)

CPU times: user 1.86 s, sys: 4.11 ms, total: 1.86 s
Wall time: 1.88 s


In [ ]:
fiona.listlayers('../../testing/Bambi.gpkg')

['Boundary', 'Points', 'Breaklines', 'layer_styles']

In [ ]:
with fiona.open('../../testing/Bambi.gpkg', layer='layer_styles') as source:
    pprint(source.schema)
    print()
    print(len(source))
    for item in source:
        props = dict(item.properties)
        #pprint(props)

{'geometry': 'None',
 'properties': {'description': 'str',
                'f_geometry_column': 'str:256',
                'f_table_catalog': 'str:256',
                'f_table_name': 'str:256',
                'f_table_schema': 'str:256',
                'owner': 'str:30',
                'styleName': 'str:30',
                'styleQML': 'str',
                'styleSLD': 'str',
                'ui': 'str:30',
                'update_time': 'datetime',
                'useAsDefault': 'int'}}

3


In [ ]:
%%time
print(bambi.boundary_estimate(perc=95))
print()

39.81252590392752

CPU times: user 856 ms, sys: 27.8 ms, total: 883 ms
Wall time: 868 ms


In [ ]:
%%time

bambi.boundary(60)

CPU times: user 1.09 s, sys: 3 µs, total: 1.09 s
Wall time: 1.2 s


In [ ]:
with fiona.open(bambi.surface, layer='Points') as source:
    for feat in source:
        geom = shape(feat.geometry)
        coords = list(geom.coords)
        print('Points:    ',coords[0])
        break

with fiona.open(bambi.surface, layer='Boundary') as source:
    for feat in source:
        geom = shape(feat.geometry)
        coords = list(geom.exterior.coords)
        print('Boundary:  ',coords[0])
        break

with fiona.open(bambi.surface, layer='Breaklines') as source:
    for feat in source:
        geom = shape(feat.geometry)
        coords = list(geom.coords)
        print('Breaklines:',coords[0])
        break

Points:     (-65836.813, -2813039.956, 1262.645)
Boundary:   (-67751.364, -2809588.9, 1232.729)
Breaklines: (-64790.526, -2817014.633)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()